In [5]:
import feedparser;
import pika;
import json;
import time;
import mysql.connector;
from datetime import datetime

In [2]:
rss_url = 'https://rss.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml'
feed = feedparser.parse(rss_url)

In [3]:
output = {
    'id':feed.entries[0].id,
    'title':feed.entries[0].title,
    'url':feed.entries[0].link,
    'author':feed.entries[0].author,
    'date':feed.entries[0].published
}

In [9]:
feed

{'bozo': False,
 'entries': [{'title': 'Stocks Sink as Trump’s Tariff Threats Weigh on Confidence',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://rss.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml',
    'value': 'Stocks Sink as Trump’s Tariff Threats Weigh on Confidence'},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://www.nytimes.com/2025/03/30/business/stocks-nikkei-sp500.html'},
    {'href': 'https://www.nytimes.com/2025/03/30/business/stocks-nikkei-sp500.html',
     'rel': 'standout',
     'type': 'text/html'}],
   'link': 'https://www.nytimes.com/2025/03/30/business/stocks-nikkei-sp500.html',
   'id': 'https://www.nytimes.com/2025/03/30/business/stocks-nikkei-sp500.html',
   'guidislink': False,
   'summary': 'Stocks in Japan tumbled nearly 4 percent as investors braced for a week of market turmoil caused by an expected announcement of more tariffs.',
   'summary_detail': {'type': 'text/html',
    'languag

In [8]:
dt = datetime.strptime(output['date'], "%a, %d %b %Y %H:%M:%S %z")
print(dt)

2025-03-31 02:45:12+00:00


In [42]:
connection = pika.BlockingConnection(pika.ConnectionParameters('192.168.0.202'))
channel = connection.channel()

In [4]:
json.dumps(output)

'{"id": "https://www.nytimes.com/2025/03/30/business/stocks-nikkei-sp500.html", "title": "Stocks Sink as Trump\\u2019s Tariff Threats Weigh on Confidence", "url": "https://www.nytimes.com/2025/03/30/business/stocks-nikkei-sp500.html", "author": "Joe Rennison", "date": "Mon, 31 Mar 2025 02:45:12 +0000"}'

In [22]:
channel.basic_publish(exchange='news.rss', routing_key='nytime', body=json.dumps(output))


In [44]:
for entrie in feed.entries:
    output = {
        'id':entrie.id,
        'title':entrie.title,
        'url':entrie.link,
        'author':entrie.author,
        'date':entrie.published
    }
    channel.basic_publish(exchange='news.rss', routing_key='nytime', body=json.dumps(output))


In [52]:
conn = mysql.connector.connect(
    host="192.168.0.202",
    port=4000,
    user="root",
    password="password",
    database="news"
)

cursor = conn.cursor()


In [56]:
sql = "INSERT INTO test (name) VALUES (%s)"
val = ["Alice"]
try:
    cursor.execute(sql, val)
    conn.commit()  # Always commit after changes
except Exception as e:
    conn.rollback()
    print("database error:", e)
finally:
    cursor.close()
    conn.close()

OperationalError: 2055: Lost connection to MySQL server at '192.168.0.202:4000', system error: 10038 在一个非套接字上尝试了一个操作。